In [7]:
import pandas as pd
import numpy as np
import pm4py

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

from xgboost import XGBRegressor
import shap

Matplotlib is building the font cache; this may take a moment.


In [9]:
xes_path = "../data/BPI_Challenge_2019-3-w-after.xes"

obj = pm4py.read_xes(xes_path)
if isinstance(obj, pd.DataFrame):
    df = obj
else:
    # legacy object -> flatten to pandas
    df = pm4py.convert_to_dataframe(obj)

# Expected columns in PM4Py DataFrame
CASE_ID = "case:concept:name"
TS = "time:timestamp"
ACT = "concept:name"
RES = "org:resource" if "org:resource" in df.columns else None

In [ ]:
# Make timestampt datetime like
df[TS] = pd.to_datetime(df[TS], errors="coerce", utc=True)

In [ ]:
case_bounds = df.groupby(CASE_ID)[TS].agg(case_start="min", case_end="max")
case_bounds["case_duration"] = case_bounds["case_end"] - case_bounds["case_start"]
case_bounds["duration_days"] = case_bounds["case_duration"].dt.total_seconds() / (3600 * 24) # seconds in an hour times hours in a day

In [12]:
maybe_case_cols = [
    "(case) Company",
    "(case) Document Type",
    "(case) Item Type",
    "(case) Spend area text",
    "(case) Sub spend area text",
    "(case) Vendor",
]
case_cols = [c for c in maybe_case_cols if c in df.columns]

# first observed values per case
case_static = (
    df.sort_values(TS)
      .groupby(CASE_ID)[case_cols]
      .first()
)

In [15]:
# activity counts per case
act_counts = (
    df.pivot_table(index=CASE_ID, columns=ACT, values=TS, aggfunc="count", fill_value=0)
      .add_prefix("act_count__")
)

# resource diversity
if RES:
    res_nunique = df.groupby(CASE_ID)[RES].nunique().rename("n_unique_resources").to_frame()
else:
    res_nunique = pd.DataFrame(index=act_counts.index)

# assemble feature table
X_case = case_static.join([act_counts, res_nunique], how="outer")

# join target
data = X_case.join(case_bounds["duration_days"], how="inner").reset_index().rename(columns={CASE_ID: "case_id"})

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.histplot(data['duration_days'], bins=50, kde=True)
plt.title('Distribution of Case Duration (days)')
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x=data['duration_days'])
plt.title('Box Plot of Case Duration (days)')
plt.show()

data['duration_days'].describe()

In [ ]:
feature_cols = [c for c in data.columns if c not in ["case_id", "duration_days"]]cat_cols = [c for c in feature_cols if data[c].dtype == "object"]num_cols = [c for c in feature_cols if c not in cat_cols]pre = ColumnTransformer(    transformers=[        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),        ("num", "passthrough", num_cols),    ],    remainder="drop",)# Remove outliers - let's clip the duration to the 99th percentile# p99 = data["duration_days"].quantile(0.99)# data_clipped = data[data["duration_days"] < p99].copy()data_clipped = data.copy()X_train, X_test = train_test_split(data_clipped, test_size=0.2, random_state=42)# Log transform the target variabley_train = np.log1p(X_train["duration_days"].astype(float))y_test = np.log1p(X_test["duration_days"].astype(float))reg = XGBRegressor(    objective="reg:squarederror",    n_estimators=500,    learning_rate=0.05,    max_depth=6,    subsample=0.8,    colsample_bytree=0.8,    random_state=42,    n_jobs=8,)pipe = Pipeline([    ("pre", pre),    ("xgb", reg),])pipe.fit(X_train[feature_cols], y_train)# Inverse transform the predictions before calculating MAEpreds_log = pipe.predict(X_test[feature_cols])preds = np.expm1(preds_log)mae = mean_absolute_error(np.expm1(y_test), preds)print(f"MAE (days): {mae:.4f}")pre_fitted = pipe.named_steps["pre"]

In [ ]:
# fit preprocessor on train to get names, then transformX_test_X = pre_fitted.transform(X_test[feature_cols])# build feature names after OHEcat_feat_names = []ohe = pre_fitted.named_transformers_.get("cat", None)if ohe is not None and hasattr(ohe, "get_feature_names_out"):    cat_feat_names = ohe.get_feature_names_out(cat_cols).tolist()num_feat_names = num_colsfeat_names = cat_feat_names + num_feat_namesbooster = pipe.named_steps["xgb"]explainer = shap.TreeExplainer(booster)shap_values = explainer.shap_values(X_test_X)# global importancemean_abs_shap = np.abs(shap_values).mean(axis=0)imp = (    pd.DataFrame({"feature": feat_names, "mean_abs_shap": mean_abs_shap})      .sort_values("mean_abs_shap", ascending=False))print("Top 20 global features by mean |SHAP|:")print(imp.head(20).to_string(index=False))# per-case top drivers (first 3 rows)for i in range(min(3, X_test_X.shape[0])):    sid = X_test.iloc[i]["case_id"]    contribs = pd.Series(shap_values[i], index=feat_names).sort_values(key=np.abs, ascending=False)    print(f"Case {sid} top drivers:")    print(contribs.head(10).to_string())